# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 2:04PM,249729,10,PRF-40662,Bio-PRF,Released
1,Oct 31 2022 2:04PM,249729,10,PRF-40667,Bio-PRF,Released
2,Oct 31 2022 2:04PM,249729,10,PRF-40682,Bio-PRF,Released
3,Oct 31 2022 2:04PM,249729,10,PRF-40685,Bio-PRF,Released
4,Oct 31 2022 2:03PM,249730,16,SHL-8715952,"SHL Pharma, LLC",Released
5,Oct 31 2022 1:58PM,249728,10,8715948,Yusen – 3D Matrix,Released
6,Oct 31 2022 1:56PM,249727,10,8715937,"ClearSpec, LLC",Released
7,Oct 31 2022 1:38PM,249726,10,SOTAH0000823,"Nextsource Biotechnology, LLC",Released
8,Oct 31 2022 1:38PM,249726,10,SONSB0002005,"Nextsource Biotechnology, LLC",Released
9,Oct 31 2022 1:32PM,249725,19,N17829,"NAPP Technologies, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,249726,Released,2
27,249727,Released,1
28,249728,Released,1
29,249729,Released,4
30,249730,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249726,NaN,NaN,2.0
249727,NaN,NaN,1.0
249728,NaN,NaN,1.0
249729,NaN,NaN,4.0
249730,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249680,0.0,0.0,1.0
249684,0.0,3.0,8.0
249685,0.0,31.0,34.0
249693,0.0,0.0,1.0
249699,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249680,0,0,1
249684,0,3,8
249685,0,31,34
249693,0,0,1
249699,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249680,0,0,1
1,249684,0,3,8
2,249685,0,31,34
3,249693,0,0,1
4,249699,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249680,,,1
1,249684,,3,8
2,249685,,31,34
3,249693,,,1
4,249699,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF
4,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC"
5,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix
6,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC"
7,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC"
9,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC"
10,Oct 31 2022 1:20PM,249723,10,Yusen – 3D Matrix
11,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc."
34,Oct 31 2022 1:06PM,249685,15,"Brookfield Pharmaceuticals, LLC"
99,Oct 31 2022 1:04PM,249721,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF,,,4
1,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",,,1
2,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix,,,1
3,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC",,,1
4,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC",,,2
5,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC",,,1
6,Oct 31 2022 1:20PM,249723,10,Yusen – 3D Matrix,,,1
7,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc.",,9,14
8,Oct 31 2022 1:06PM,249685,15,"Brookfield Pharmaceuticals, LLC",,31,34
9,Oct 31 2022 1:04PM,249721,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF,4,,
1,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",1,,
2,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix,1,,
3,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC",1,,
4,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC",2,,
5,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC",1,,
6,Oct 31 2022 1:20PM,249723,10,Yusen – 3D Matrix,1,,
7,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc.",14,9,
8,Oct 31 2022 1:06PM,249685,15,"Brookfield Pharmaceuticals, LLC",34,31,
9,Oct 31 2022 1:04PM,249721,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF,4,,
1,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",1,,
2,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix,1,,
3,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC",1,,
4,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF,4.0,NaN,NaN
1,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",1.0,NaN,NaN
2,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 2:04PM,249729,10,Bio-PRF,4.0,0.0,0.0
1,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",1.0,0.0,0.0
2,Oct 31 2022 1:58PM,249728,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Oct 31 2022 1:56PM,249727,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Oct 31 2022 1:38PM,249726,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3496059,46.0,6.0,0.0
15,499369,42.0,34.0,0.0
16,749139,2.0,1.0,2.0
19,249725,1.0,0.0,0.0
20,499402,15.0,9.0,0.0
21,998846,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3496059,46.0,6.0,0.0
1,15,499369,42.0,34.0,0.0
2,16,749139,2.0,1.0,2.0
3,19,249725,1.0,0.0,0.0
4,20,499402,15.0,9.0,0.0
5,21,998846,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,6.0,0.0
1,15,42.0,34.0,0.0
2,16,2.0,1.0,2.0
3,19,1.0,0.0,0.0
4,20,15.0,9.0,0.0
5,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,15,Released,42.0
2,16,Released,2.0
3,19,Released,1.0
4,20,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,2.0,0.0,0.0,0.0
Executing,6.0,34.0,1.0,0.0,9.0,0.0
Released,46.0,42.0,2.0,1.0,15.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,6.0,34.0,1.0,0.0,9.0,0.0
2,Released,46.0,42.0,2.0,1.0,15.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,6.0,34.0,1.0,0.0,9.0,0.0
2,Released,46.0,42.0,2.0,1.0,15.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()